# 💵 Détection de faux billets

In [ ]:
pip install pca

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.cluster import AgglomerativeClustering
from sklearn.ensemble import RandomForestClassifier
import scipy.cluster.hierarchy as shc
from pca import pca

In [ ]:
# Couleurs
red = '#d50000'
teal = '#00bfa5'
indigo = '#304ffe'
amber = '#ffab00'
purple = '#aa00ff'

# Données

In [ ]:
source = 'https://raw.githubusercontent.com/gllmfrnr/oc/master/p6/notes.csv'
df = pd.read_csv(source)
df

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# Nombre de duplicats
print(len(df) - len(df.drop_duplicates()), 'duplicat')

In [ ]:
# Variables supplémentaires pour l'analyse
df['ratio_heights'] = df['height_left'] / df['height_right']
df['ratio_margins'] = df['margin_up'] / df['margin_low']
df.sample(3)

# Analyse des corrélations

In [ ]:
# Répartition des vrais et faux billets pour chaque variable
for i in df.columns[1:]:
    sns.boxplot(data=df, x='is_genuine', y=i)
    title = 'Répartition de ' + i
    plt.title(title)
    plt.show()
    
# --> Les vrais billets ont des hauteurs et des marges plus petites que les faux
# --> Les vrais villets sont plus longs que les faux
# --> Les diagonales des vrais et des faux billets sont similaires (légèrement supérieures chez les vrais)

In [ ]:
# Scatterplot entre la longueur et les autres quantitatives
sns.scatterplot(data=df, x='length', y='margin_up', hue='is_genuine')
plt.show()
sns.scatterplot(data=df, x='length', y='height_right', hue='is_genuine')
plt.show()
sns.scatterplot(data=df, x='length', y='margin_low', hue='is_genuine')
plt.show()
sns.scatterplot(data=df, x='length', y='height_left', hue='is_genuine')
plt.show()

# --> Les vrais billets sont les plus longs
# --> Les vrais billets concentrent des hauteurs et marges légèrement inférieures aux faux billets

In [ ]:
# Scatterplot entre la diagonale et les autres quantitatives
sns.scatterplot(data=df, x='diagonal', y='margin_up', hue='is_genuine')
plt.show()
sns.scatterplot(data=df, x='diagonal', y='height_right', hue='is_genuine')
plt.show()
sns.scatterplot(data=df, x='diagonal', y='margin_low', hue='is_genuine')
plt.show()
sns.scatterplot(data=df, x='diagonal', y='height_left', hue='is_genuine')
plt.show()

# --> La distribution des diagonales des vrais et faux billets est semblable
# --> Les vrais billets concentrent encore les valeurs de dimensions les plus faibles

In [ ]:
sns.boxplot(data=df, x='is_genuine', y='ratio_heights')
plt.show()

# --> Les faux billets ont un ratio de hauteurs gauche et droite plus proche de 1 que les vrais billets

In [ ]:
sns.lmplot(data=df, x='height_right', y='height_left', hue='is_genuine')
plt.show()

In [ ]:
sns.lmplot(data=df, x='margin_low', y='margin_up', hue='is_genuine')
plt.show()

# --> Les faux billets ont des ratios de marges plus grand que les vrais billets

In [ ]:
plt.figure(figsize=(16, 6))
mask = np.triu(np.ones_like(df.corr(), dtype=np.bool))
heatmap = sns.heatmap(df.corr(), mask=mask, vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Triangle correlation heatmap', fontdict={'fontsize':16}, pad=16)
plt.show()

# --> La véracité du billet est corrélée positivement à la longueur, et négativement à la marge inférieure

# ACP

In [ ]:
# Projection sur les variables à normaliser
variables_a_normaliser = df.drop([
    'is_genuine',
    'ratio_heights',
    'ratio_margins'], axis=1)

# Normalisation des variables
variables = normalize(variables_a_normaliser.values)
variables = pd.DataFrame(variables, columns=variables_a_normaliser.columns)
variables

In [ ]:
# ACP avec le module pca
acp = PCA().fit(variables)
acp

# Plotting the Cumulative Summation of the Explained Variance
plt.figure()
plt.plot(np.cumsum(acp.explained_variance_ratio_))
plt.xlabel('Composantes')
plt.ylabel('Variance Explained (%)') # For each component
plt.title('Pulsar Dataset Explained Variance')
plt.show()

In [ ]:
# Modèle
model = pca(n_components=2)
results = model.fit_transform(variables)
results

# --> Les topfeats confirment que 'length' et 'margin_low' sont les meilleures explicatives

In [ ]:
fig, ax = model.plot()

In [ ]:
# Jointure des clusters sur les composantes de chaque individu
deux_composantes = pd.concat([results['PC'], df['is_genuine']], axis=1)
deux_composantes

In [ ]:
# Scatterplot des 2 composantes
plt.figure(figsize=(8,5))
sns.scatterplot(
    data=deux_composantes, x='PC1', y='PC2', hue='is_genuine', 
    palette=[indigo, amber], s=100)
plt.title('Représentation des 2 composantes, par véracité')
plt.show()

In [ ]:
# Dendrogramme
plt.figure(figsize=(15, 5))  
plt.title('Dendrogramme')  
dend = shc.dendrogram(shc.linkage(results['PC'], method='ward'))

# Threshold
plt.axhline(y=.04, color='r', linestyle='--')
plt.show()

In [ ]:
# Cluster de chaque individu
cluster = AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='ward')  
clusters = pd.DataFrame(cluster.fit_predict(variables), columns=['cluster'])
df = df.merge(clusters, left_index=True, right_index=True)
df = df.merge(results['PC'], left_index=True, right_index=True)
df

In [ ]:
# Effectif pour vrai ou faux
sns.countplot(data=df, x='is_genuine', hue='cluster')
plt.title('Effectif des vrais et faux billets, par cluster')
plt.show()

# --> A deux exceptions près, les clusters correspondent exactement à vrai ou faux

In [ ]:
# Scatterplot des 2 composantes, par cluster
plt.figure(figsize=(8,5))
sns.scatterplot(
    data=df, x='PC1', y='PC2', hue='cluster', 
    palette=[amber, indigo], s=400)
plt.title('Représentation des 2 composantes, par cluster')
sns.scatterplot(
    data=df, x='PC1', y='PC2', hue='is_genuine', 
    palette=[teal, purple], s=100)
plt.title('Représentation des 2 composantes, par cluster')
plt.show()

# --> 2 faux billets ont été intégrés au cluster des vrais billets
# --> Quelque soit le nombre de composantes, avec une variance expliquée de 100%, les 2 faux billets apparaissent

# 2 outliers

In [ ]:
# Affichage des 2 individus non détectés par les clusters
outliers = df[(df['cluster']==0) & (df['is_genuine']==True)]
outliers

In [ ]:
df['outlier'] = 'Détecté'
df['outlier'].loc[outliers.index] = 'Non détecté'
df

In [ ]:
sns.boxplot(data=df[df['is_genuine']==True], y='outlier', x='length')
plt.show()

In [ ]:
sns.boxplot(data=df[df['is_genuine']==True], y='outlier', x='margin_low')
plt.show()

In [ ]:
sns.scatterplot(data=df[df['is_genuine']==False], y='length', x='margin_low', hue='cluster', s=100)
plt.show()

In [ ]:
sns.scatterplot(data=df[df['is_genuine']==True], y='length', x='margin_low', hue='cluster', s=100)
plt.show()

# Modèle

In [ ]:
df.sample()

In [ ]:
def modele(data):
    features = ['length', 'margin_low']
    X_test = pd.DataFrame([[110.0, 3.0], [122.0, 6.0], [119.2, 4.9], [116.2, 3.9]], columns=features)
    X = df[features]
    y = df['is_genuine']
    model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
    model.fit(X, y)
    predictions = model.predict(X_test)
    output = pd.concat([X_test, pd.DataFrame(predictions, columns=['prediction'])], axis=1)
    return output
    
modele(df)

In [ ]:
sns.scatterplot(data=df.sample(12), x='length', y='margin_low', hue='is_genuine', s=250)
sns.scatterplot(data=output, x='length', y='margin_low', hue='prediction', s=2500)
plt.show